# **Tarea Nº 4**

- Fecha y hora de entrega: Domingo 12 de enero, 11:59pm
- Agregue los nombres de las personas con las que discutió esta tarea: __Rodrigo Farfan__
- Envíe su tarea haciendo el `push` de su código a su repo en GitHub Classroom: 

## **Mercado de autos usados**

Importa el conjunto de datos `neoauto_20240924.csv`
    

1. ¿Cuál es el precio promedio de los autos de cada marca (`item_brand`) publicados en cada subcategoría (`item_category_2`)?

In [7]:
# Solución

import pandas as pd

dfautos = pd.read_csv("neoauto_20240924.csv")

promedio_precio = dfautos.groupby(['item_brand', 'item_category_2'])['item_price'].mean().reset_index()

# Mostrar la información obtenida en un dataframe nuevo
print(promedio_precio)

       item_brand item_category_2     item_price
0      ALFA-ROMEO       Hatchback    8500.000000
1      ALFA-ROMEO           Sedan    2500.000000
2    ASTON MARTIN  Camionetas Suv  229000.000000
3    ASTON MARTIN       Deportivo  169990.000000
4    ASTON MARTIN           Sedan  149900.000000
..            ...             ...            ...
197         VOLVO  Camionetas Suv   41801.707317
198         VOLVO       Deportivo    8800.000000
199         VOLVO       Hatchback   12772.500000
200         VOLVO           Sedan   11458.333333
201         VOLVO   Station Wagon    6100.000000

[202 rows x 3 columns]


2. ¿Cuántos autos tienen más de 100,000 kilómetros (`item_km`) pero un precio (`item_price`) menor al promedio general?

In [9]:
# Solución

precio_promedio = dfautos['item_price'].mean() # Promedio general

autos_filtrados = dfautos[(dfautos['item_km'] > 100000) & (dfautos['item_price'] < precio_promedio)] # Haciendo el filtro

numero_autos = autos_filtrados.shape[0] # Hallando el número

print(numero_autos)

497


3. Encuentra el modelo (`item_name`) más caro y el más barato de cada marca (`item_brand`).

In [11]:
# Solución

# Hallando los máximos y mínimos de cada marca y anexándolos a nuevos dataframe
max_precio = dfautos.loc[dfautos.groupby('item_brand')['item_price'].idxmax()][['item_brand', 'item_name', 'item_price']]
min_precio = dfautos.loc[dfautos.groupby('item_brand')['item_price'].idxmin()][['item_brand', 'item_name', 'item_price']]

# Unir los dataframes para obtener el resultado final
resultados = pd.merge(max_precio, min_precio, on='item_brand', suffixes=('_max', '_min'))

print(resultados)


      item_brand             item_name_max  item_price_max  \
0     ALFA-ROMEO       alfa-romeo 147 2006          8500.0   
1   ASTON MARTIN     aston martin dbx 2021        229000.0   
2           AUDI            audi rs 6 2024        220000.0   
3         AUSTIN   austin mini cooper 1970             1.0   
4           BAIC             baic x55 2021         12800.0   
..           ...                       ...             ...   
68           SWM          swm g05 pro 2022         14000.0   
69        TOYOTA         toyota hilux 2019        100000.0   
70       TRIUMPH         triumph tr-7 1971         18900.0   
71    VOLKSWAGEN  volkswagen teramont 2023         48900.0   
72         VOLVO          volvo xc-90 2023         84990.0   

                     item_name_min  item_price_min  
0          alfa-romeo alfasud 1982          2500.0  
1           aston martin db11 2017        149900.0  
2               audi a3 sedan 2015             0.0  
3          austin mini cooper 1970         

4. Encuentra las tres marcas con mayor número de autos financiados por Santander (`item_financed_by`). 

In [12]:
# Solución

autos_financiados_santander = dfautos[dfautos['item_financed_by'] == 'Santander']

conteo_marcas = autos_financiados_santander['item_brand'].value_counts()

print(conteo_marcas.head(3))

item_brand
BMW       296
TOYOTA    261
NISSAN    228
Name: count, dtype: int64


5. Agrupa las publicaciones por tipo de transmisión (`item_transmission`) y calcula la desviación estándar del precio (`item_price`) para cada tipo.  

In [15]:
# Solución

dfautos.groupby('item_transmission')['item_price'].std().reset_index().rename(columns={'item_price': 'sd_itemprice'})

,item_transmission,sd_itemprice
0,Automática,15332.729048
1,Automática - Secuencial,22678.641589
2,Mecánica,8137.122127


6. ¿Cuál es el kilometraje promedio de las publicaciones con el tag "Premium" y de las publicaciones con el tag "Como nuevo"?

In [17]:
# Solución
dfautos[dfautos['item_tag'].isin(['Premium', 'Como nuevo'])].groupby('item_tag')['item_km'].mean().reset_index()

,item_tag,item_km
0,Como nuevo,35963.476190
1,Premium,48400.235897


7. ¿Cuáles son las marcas de vehículas para los que todos sus modelos fabricados en los últimos 5 años tienen al menos 5 publicaciones cada una?

In [20]:
# Solución

from datetime import datetime

anio_actual = datetime.now().year

dfautos_recientes = dfautos[dfautos['item_year'] >= anio_actual - 5]

conteo_publicaciones = dfautos_recientes.groupby(['item_brand', 'item_name']).size()

modelos_filtrados = conteo_publicaciones[conteo_publicaciones >= 5]

marcas_validas = []
for marca, grupo in dfautos_recientes.groupby('item_brand'):
    
    modelos_marca = grupo['item_name'].unique()
    publicaciones_modelos = conteo_publicaciones[conteo_publicaciones.index.get_level_values('item_brand') == marca]
    
    if all(publicaciones_modelos >= 5):
        marcas_validas.append(marca)

print(marcas_validas)

[]


## **Evolución económica internacional**

Importa el conjunto de datos de la hoja "Data" del archivo `pwt1001.xlsx`. El diccionario de variables está disponible en la hoja "Legend" del archivo Excel.
    

1. Calcule el PBI per cápita real ajustado por poder de paridad de compra (PPP). Use la variable de PBI que se calcula por gasto y la población. ¿Cuáles son los 10 países top?

In [ ]:
# Solución

2. Genera el ránking de los países con el mayor incremento del PBI per cápita real PPP entre los años 1990 y 2019. ¿Dónde se ubica Perú? Para los países que no tengan información para dichos años, puede utilizar datos provenientes de hasta dos años antes o después

In [ ]:
# Solución

3. Para el periodo 2010-2019, calcule el promedio del índice del capital humano. ¿Qué relación existe entre esta variable y el PBI per cápita real PPP calculado previamente? ¿y el número de horas laboradas al año por los trabajadores?

In [ ]:
# Solución

4. A partir de la variable del precio de los hogares, calcula los países y los años en los cuales se registró la mayor inflación anual para todo el periodo de tiempo disponible en la base de datos

In [ ]:
# Solución

5. ¿Cuáles son los países con la mayor cantidad de missing values en la variable de productividad total de factores a precios corrientes?

In [ ]:
# Solución

6.  Agrupa los datos por país y calcula el promedio de las horas trabajadas anuales (avh) y de la educación promedio (hc) para el periodo 2000-2020.

In [ ]:
# Solución

7.  Genere una variable que ordene a los países según el porcetaje de formación brutal de capital del 2019. Luego, estable diez grupos a manera de deciles. ¿En qué decil se ubica Péru? Vuelva a calcular usando la información de 1990. ¿El Perú se mantiene en el mismo décil o cambia?

In [ ]:
# Solución

## **Transacciones Financieras**

Importa el conjunto de datos `base_financiera.csv`
    

8. Filtra todas las transacciones de tipo "Debit" en las que el monto sea mayor a 200, y calcula el monto total de esas transacciones. Además, determina cuántos clientes únicos realizaron esas transacciones.

In [10]:
# Solución

import pandas as pd

dffinance = pd.read_csv("base_financiera.csv")

transacciones_debit = dffinance[(dffinance['Transaction_Type'] == 'Debit') & (dffinance['Amount'] > 200)]

# Calculamos el monto total de esas transacciones
monto_total = transacciones_debit['Amount'].sum()

# Determinamos cuántos clientes únicos realizaron esas transacciones
clientes_unicos = transacciones_debit['Customer_ID'].nunique()

# Mostrar los resultados
print(f'Monto total de las transacciones: {monto_total}')
print(f'Número de clientes únicos: {clientes_unicos}')

Monto total de las transacciones: 2847871.75
Número de clientes únicos: 200


9. Identifica las transacciones con valores nulos en la columna (`Amount`) . Rellena esos valores con la mediana de los montos por tipo de transacción (`Transaction_Type`) y luego calcula el monto total de transacciones por región (`region`) .

In [13]:
# Solución

import pandas as pd

transacciones_nulas = dffinance[dffinance['Amount'].isnull()]

medianas_por_tipo = dffinance.groupby('Transaction_Type')['Amount'].median()

# Función para rellenar los valores nulos con la mediana correspondiente
def rellenar_nulos(row):
    if pd.isnull(row['Amount']):
        return medianas_por_tipo[row['Transaction_Type']]
    else:
        return row['Amount']

dffinance['Amount'] = dffinance.apply(rellenar_nulos, axis=1)

monto_total_por_region = dffinance.groupby('Region')['Amount'].sum().reset_index()

print("\nMonto total de transacciones por región:")
print(monto_total_por_region)


Monto total de transacciones por región:
  Region      Amount
0   East  1269409.53
1  North  1321224.96
2  South  1208300.23
3   West  1233552.50


10. Busca transacciones duplicadas considerando las columnas (`Customer_ID`) y (`Date`). Elimina los duplicados manteniendo solo la primera ocurrencia. Luego, agrupa las transacciones restantes por (`Customer_ID`) y calcula el monto promedio y la cantidad de transacciones por cliente.

In [ ]:
# Solución